**Simple TEXT CLASSIFICATION**


**STEP 1**: *Unstructured to Structured Data*

In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
import pandas, xgboost, numpy, textblob, string

In [177]:
# load the dataset
data = open('/corpus').read()
labels, texts = [], []
#flag=0
for i, line in enumerate(data.split("\n")):                                      # read the data load it and segregate w.r.t space and divide & assign the content array into two called labels and text.
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))
    if(i<7):                                                                     # to view the corpus file data before dataframing sample 7
      print('Label:', labels[i],"\n",'Text:', content[1:])
       
'''
    if(flag==0):
      print('Label:', labels[i],"\n",'Text:', content[1:])
      flag=1 
'''  

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

print ("\n",trainDF)                                                             #complete dataframe every row and column matrix format
print("\n",trainDF.iloc[3])                                                      #row vise selection in dataframe

Label: __label__2 
 Text: ['Stuning', 'even', 'for', 'the', 'non-gamer:', 'This', 'sound', 'track', 'was', 'beautiful!', 'It', 'paints', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'I', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid.', 'game', 'music!', 'I', 'have', 'played', 'the', 'game', 'Chrono', 'Cross', 'but', 'out', 'of', 'all', 'of', 'the', 'games', 'I', 'have', 'ever', 'played', 'it', 'has', 'the', 'best', 'music!', 'It', 'backs', 'away', 'from', 'crude', 'keyboarding', 'and', 'takes', 'a', 'fresher', 'step', 'with', 'grate', 'guitars', 'and', 'soulful', 'orchestras.', 'It', 'would', 'impress', 'anyone', 'who', 'cares', 'to', 'listen!', '^_^']
Label: __label__2 
 Text: ['The', 'best', 'soundtrack', 'ever', 'to', 'anything.:', "I'm", 'reading', 'a', 'lot', 'of', 'reviews', 'saying', 'that', 'this', 'is', 'the', 'best', "'game", "soundtrack'", 'and', 'I', 'figured', 'that', "I'd", 'write', 'a', 'review', 'to', 'disagree', 'a', 'bit.', 'This', 'in

In [178]:
# split the dataset into training and validation datasets randomly >>> x represents text dataframe and y => label dataframe 
#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.40, random_state=42)



print("Training set of text train_x""\n")
print(train_x.iloc[3])                                                           # to check training set of text
print("\n""respective training set of label train_y""\n")
print(train_y.iloc[3])                                                           # to check the training set of label

# label encode the target variable i.e convert the labels into 0 to n-class-1
encoder = preprocessing.LabelEncoder()                                           # 'encoder' is the object
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#encoded results of 7 labels
print(train_y[0:7])


#In order to reverse the encoded format of the train_y and valid_y

#list(encoder.inverse_transform(train_y))



Training set of text train_x

Digital Copy Doesn't Work: Bought this after the Digital Copy expiration date had elapsed. Nothing on the Amazon site indicates the expiration date. The package has this warning on both the outside and inside. Amazon offered a full refund. That's a stand up company in my opinion.

respective training set of label train_y

__label__1
[0 0 0 0 1 0 0]


**STEP 2**: ***Feature Selection***

 ***Count vectors features***

In [179]:
# create a count vectorizer object 
count_vector_object = CountVectorizer(analyzer='word', token_pattern=r'\w+')     # word level tokens as per RegExp
#xtrain_count =  count_vector_object.fit_transform(train_x)                      # future we will get dimensionality error in classifier
#xvalid_count =  count_vector_object.fit_transform(valid_x)

count_vector_object.fit(trainDF['text'])                                         # seperate fit and transform can be applied but fit_transform is best

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vector_object.transform(train_x)
xvalid_count =  count_vector_object.transform(valid_x)

print(count_vector_object.get_feature_names())
#print(count_vector_object.get_params())                                         # count vector object parameters
print(xtrain_count)                                                              # O/P Details::
print(xvalid_count)                                                              # (0[row_number], feature_column_number) frequency_count_of_feature


#print(numpy.matrix(xvalid_count))
#print(numpy.array(xtrain_count).size)

['0', '00', '000', '001', '002', '00290', '007', '0070412901', '0072316373', '008', '00now', '00yeah', '01', '011', '02', '03', '04', '05', '052', '06', '06rate', '07', '088', '09', '0s9', '0sx', '1', '10', '100', '1000', '1000amp', '1000s', '1000uf', '1001', '100m', '100th', '100this', '101', '10162', '102', '1020', '1021', '103', '1030pm', '104', '1048259', '105', '1058', '1059', '107', '108', '1080p', '10bm', '10ft', '10gameplay', '10gb', '10i', '10mb', '10min', '10movie', '10overall', '10p', '10sound', '10th', '10the', '10this', '11', '110', '1100', '110e', '1115', '11223', '114622', '115', '116', '119', '11b', '11g', '11i', '11lbs', '11movie', '11th', '12', '120', '1200', '120100', '120lbs', '120s', '120v', '121', '1215', '123', '125', '128', '128mb', '129', '12th', '12x', '13', '130', '133x', '1340s', '135lbs', '1394', '13th', '13w', '13x', '14', '140', '140lbs', '144', '145', '14th', '14yr', '15', '150', '15000', '1537', '158', '15hrs', '15sci_fi_researcher', '15th', '16', '160'

**STEP 3: MODEL BUILDING and ACCURACY FINDING**


In [180]:
# function to train the model
def train_test_model(classifier, feature_vector_train, label, feature_vector_valid, name):
                                                                                 # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
                                                                                 # save the trained model in the ".pkl" file so that objects are serialised into the pickle file.
    joblib.dump(classifier, '/' + name + '.pkl') 

                                                                                 # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return metrics.accuracy_score(predictions, valid_y)


# Naive Bayes on Count Vectors
accuracy = train_test_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count,'NB-Count')
print("NB, Count Vectors: ", accuracy*100)


NB, Count Vectors:  83.0


# ***Info of  Output ***:
**Standard deviation for 20 results = 0.7035.** 
 **Mean value for the 20 results = 83.66**
* why average result?
dataset division as train and test is random, hence result accuracy changes. average value calculated.

**Results of Fixed percentage of test and train set**
* Testset =25% and Trainset=75% acc= 83.20%
* Testset =30% and Trainset=70% acc= 83.06%
* Testset =20% and Trainset=80% acc= 82.89%
* Testset =40% and Trainset=60% acc= 83.00%